In [286]:
import requests
import pandas as pd

years = range(2005, 2016)  # From 2010 to 2022
data = []

# Replace with your API key
API_KEY = '59951fa51ae0813c57d78e2f22413294efb566a6'
variable_map = {
    2005: ['B01001_001E', 'B19013_001E', 'DP03_0003E', 'DP03_0003E', 'B01002_001E'],
    2006: ['B01001_001E', 'B19013_001E', 'B23025_005E', 'B23025_003E', 'B01002_001E'],
    2007: ['B01001_001E', 'B19013_001E', 'B23025_005E', 'B23025_003E', 'B01002_001E'],
    2008: ['B01001_001E', 'B19013_001E', 'B23025_005E', 'B23025_003E', 'B01002_001E'],
    2009: ['B01001_001E', 'B19013_001E', 'B23025_005E', 'B23025_003E', 'B01002_001E'],
    2010: ['B01001_001E', 'B19013_001E', 'B23025_005E', 'B23025_003E', 'B01002_001E'],
    2011: ['B01001_001E', 'B19013_001E', 'B23025_005E', 'B23025_003E', 'B01002_001E'],
    2012: ['B01001_001E', 'B19013_001E', 'B23025_005E', 'B23025_003E', 'B01002_001E'],
    2013: ['B01001_001E', 'B19013_001E', 'B23025_005E', 'B23025_003E', 'B01002_001E'],
    2014: ['B01001_001E', 'B19013_001E', 'B23025_005E', 'B23025_003E', 'B01002_001E'],
    2015: ['B01001_001E', 'B19013_001E', 'B23025_005E', 'B23025_003E', 'B01002_001E']
}
# DP03_0003E, DP03_0005E
for year in years:
    if year > 2010:
        # For years greater than 2010, make a single API call to get all data
        url = f"https://api.census.gov/data/{year}/acs/acs1?get=NAME,B01001_001E,B19013_001E,B23025_005E,B23025_003E,B01002_001E&for=place:*&key={API_KEY}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data for year {year}")
            continue
        json_data = response.json()
        for row in json_data[1:]:  # Skip the header row
            city_name = row[0]
            try:
                population = int(row[1])
                mean_income = float(row[2])
                unemployed = float(row[3])
                total_labor_force = float(row[4])
                median_age = float(row[5])
            except:
                population = 0

            # Filter for population greater than 100,000
            if population > 100000:
                data.append({
                    'Year': year,
                    'City': city_name,
                    'Total Population': population,
                    'Mean Household Income': mean_income,
                    'Unemployment Rate': unemployed/total_labor_force*100,
                    'Median Age': median_age
                })
    else:
        # For years <= 2010, make two API calls: one for general data and one for unemployment data
        url_general = f"https://api.census.gov/data/{year}/acs/acs1?get=NAME,B01001_001E,B19013_001E,B01002_001E&for=place:*&key={API_KEY}"
        url_unemployment = f"https://api.census.gov/data/{year}/acs/acs1/profile?get=NAME,DP03_0003E,DP03_0005E&for=place:*&key={API_KEY}"
        
        response_general = requests.get(url_general)
        response_unemployment = requests.get(url_unemployment)
        
        if response_general.status_code != 200 or response_unemployment.status_code != 200:
            print(f"Failed to fetch data for year {year}")
            continue
        
        json_data_general = response_general.json()
        json_data_unemployment = response_unemployment.json()
        
        # Create a lookup for unemployment data by city name
        unemployment_dict = {}
        for row in json_data_unemployment[1:]:  # Skip the header row
            city_name = row[0]
            try:
                unemployed = float(row[1])
                total_labor_force = float(row[2])
                unemployment_dict[city_name] = (unemployed, total_labor_force)
            except:
                unemployment_dict[city_name] = (0, 0)
        
        for row in json_data_general[1:]:  # Skip the header row
            city_name = row[0]
            try:
                population = int(row[1])
                mean_income = float(row[2])
                median_age = float(row[3])
                unemployed, total_labor_force = unemployment_dict.get(city_name, (0, 0))
            except:
                population = 0

            # Filter for population greater than 100,000
            if population > 100000:
                if total_labor_force > 0:  # Prevent division by zero
                    unemployment_rate = unemployed / total_labor_force 
                else:
                    unemployment_rate = None
                
                data.append({
                    'Year': year,
                    'City': city_name,
                    'Total Population': population,
                    'Mean Household Income': mean_income,
                    'Unemployment Rate': unemployment_rate,
                    'Median Age': median_age
                })

# Create a DataFrame
df = pd.DataFrame(data)
state_abbreviation_map = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI',
    'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Step 2: Replace state names with abbreviations in the 'City' column
def replace_state_names(city_name):
    for state, abbrev in state_abbreviation_map.items():
        if state in city_name:
            return city_name.replace(state, abbrev)
    return city_name
pivot_income = df.pivot(index='City', columns='Year', values='Mean Household Income')
df_income = pivot_income
df_income = df_income.dropna()
# cleaning out stuff
df_income.reset_index(inplace=True)
df_income = df_income[~df_income['City'].str.contains("Puerto Rico", case=False, na=False)]
df_income['City'] = df_income['City'].str.replace(r' \bcity\b', '', regex=True)
df_income['City'] = df_income['City'].str.replace(r' \bmunicipality\b', '', regex=True)
df_income['City'] = df_income['City'].str.replace(r' \bCDP\b', '', regex=True)
df_income['City'] = df_income['City'].str.strip()
df_income['City'] = df_income['City'].apply(replace_state_names)
df_income.reset_index(inplace=True)
df_income = df_income.drop(columns=['index'], errors='ignore')
df_income


Year,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,"Abilene, TX",35246.0,37578.0,38502.0,38968.0,41957.0,40244.0,38403.0,41197.0,43059.0,43198.0,44176.0
1,"Akron, OH",32937.0,31566.0,32928.0,32499.0,32892.0,31171.0,31392.0,32645.0,34300.0,34781.0,34639.0
2,"Albuquerque, NM",41820.0,43021.0,43677.0,46437.0,44594.0,46079.0,43606.0,45373.0,48357.0,46433.0,47096.0
3,"Alexandria, VA",66116.0,80449.0,80806.0,86682.0,77095.0,77793.0,82748.0,81160.0,86775.0,86809.0,90056.0
4,"Allentown, PA",33658.0,34834.0,36074.0,39036.0,33664.0,32192.0,30784.0,36158.0,36655.0,39549.0,37146.0
...,...,...,...,...,...,...,...,...,...,...,...,...
229,"West Valley City, UT",43981.0,46817.0,49691.0,53662.0,51510.0,51421.0,47679.0,48518.0,51027.0,56037.0,57111.0
230,"Wichita, KS",40115.0,41089.0,42696.0,44211.0,44405.0,42694.0,44184.0,43776.0,43538.0,45477.0,46894.0
231,"Winston-Salem, NC",38197.0,38594.0,39589.0,40869.0,39808.0,40401.0,37501.0,37592.0,37322.0,40869.0,41287.0
232,"Worcester, MA",37797.0,43779.0,45058.0,44890.0,47415.0,42887.0,42718.0,43492.0,45011.0,46449.0,43572.0


In [287]:
pivot_unem = df.pivot(index='City', columns='Year', values='Unemployment Rate')
df_unem = pivot_unem
df_unem = df_unem.dropna()
# cleaning out stuff
df_unem.reset_index(inplace=True)
df_unem = df_unem[~df_unem['City'].str.contains("Puerto Rico", case=False, na=False)]
df_unem['City'] = df_unem['City'].str.replace(r' \bcity\b', '', regex=True)
df_unem['City'] = df_unem['City'].str.replace(r' \bmunicipality\b', '', regex=True)
df_unem['City'] = df_unem['City'].str.replace(r' \bCDP\b', '', regex=True)
df_unem['City'] = df_unem['City'].str.strip()
df_unem['City'] = df_unem['City'].apply(replace_state_names)
df_unem.reset_index(inplace=True)
df_unem = df_unem.drop(columns=['index'], errors='ignore')
df_unem

Year,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,"Abilene, TX",13.694763,13.517305,29.180902,31.096831,19.354804,11.067088,6.262301,7.962637,5.589826,5.513243,4.721397
1,"Akron, OH",10.318200,10.303280,9.444551,7.881542,5.909530,6.161665,16.665995,16.642479,13.389225,8.800773,8.831826
2,"Albuquerque, NM",14.533702,18.786019,19.911130,19.150972,13.877965,12.436434,10.542100,8.817330,7.260191,8.308843,6.681222
3,"Alexandria, VA",23.768228,40.474035,29.545519,24.016289,16.468271,23.719207,6.362097,4.413465,4.519605,3.842083,3.440521
4,"Allentown, PA",9.858149,10.491041,10.938176,11.420284,6.481526,6.079585,16.842668,12.230240,16.841752,14.824306,10.392898
...,...,...,...,...,...,...,...,...,...,...,...,...
224,"West Valley City, UT",15.081935,22.825581,19.586083,23.797096,11.003100,8.787467,10.190363,11.420777,7.848644,7.139091,4.553864
225,"Wichita, KS",12.318049,13.096823,15.245344,16.408939,9.897806,8.734963,10.462393,8.034788,7.482116,6.848021,4.994331
226,"Winston-Salem, NC",14.281379,16.299441,18.800324,13.329538,7.653060,9.649404,13.692842,13.601368,8.911535,12.036007,8.199884
227,"Worcester, MA",12.030378,14.151632,20.314155,12.681598,8.478956,7.948234,10.230195,11.720102,8.584090,9.296593,8.695003


In [288]:
pivot_age = df.pivot(index='City', columns='Year', values='Median Age')
df_age = pivot_age
df_age = df_age.dropna()
# cleaning out stuff
df_age.reset_index(inplace=True)
df_age = df_age[~df_age['City'].str.contains("Puerto Rico", case=False, na=False)]
df_age['City'] = df_age['City'].str.replace(r' \bcity\b', '', regex=True)
df_age['City'] = df_age['City'].str.replace(r' \bmunicipality\b', '', regex=True)
df_age['City'] = df_age['City'].str.replace(r' \bCDP\b', '', regex=True)
df_age['City'] = df_age['City'].str.strip()
df_age['City'] = df_age['City'].apply(replace_state_names)
df_age.reset_index(inplace=True)
df_age = df_age.drop(columns=['index'], errors='ignore')
df_age

Year,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,"Abilene, TX",32.3,32.7,31.9,31.7,31.5,32.7,32.5,31.8,30.9,31.3,31.3
1,"Akron, OH",34.7,35.5,35.5,35.7,36.0,34.6,36.6,35.1,36.4,37.9,38.1
2,"Albuquerque, NM",35.8,35.3,35.2,35.5,34.0,35.2,35.5,35.3,35.8,35.7,36.0
3,"Alexandria, VA",36.2,37.0,37.5,38.1,35.1,34.9,35.7,35.8,36.0,36.1,36.6
4,"Allentown, PA",34.2,33.1,32.2,33.1,32.3,33.7,32.3,33.0,32.7,32.3,33.5
...,...,...,...,...,...,...,...,...,...,...,...,...
229,"West Valley City, UT",27.7,29.7,28.2,30.3,30.4,30.3,29.8,30.6,30.4,29.7,30.7
230,"Wichita, KS",34.2,34.8,35.0,34.0,34.3,33.9,34.3,35.1,34.0,34.0,34.6
231,"Winston-Salem, NC",34.3,34.4,36.0,34.5,35.0,33.8,34.8,34.4,35.1,35.2,34.7
232,"Worcester, MA",34.9,33.3,34.2,35.2,33.6,34.0,33.8,32.7,32.7,33.0,34.9


In [289]:
crimedata = pd.read_csv("../data/crime_rate.csv") 

In [290]:
uniquecities = crimedata["City, State"].unique()
df_income_cr = df_income[df_income["City"].isin(uniquecities)]
df_unem_cr = df_unem[df_unem["City"].isin(uniquecities)]
df_age_cr = df_age[df_age["City"].isin(uniquecities)]

In [291]:
crimedata_filter = crimedata[crimedata["Year"] >=2005]

In [292]:
crimedata_filter

,Year,index_nsa,"City, State",Population,Violent Crimes,Homicides,Rapes,Assaults,Robberies
2510,2005.0,171.870000,"Albuquerque, NM",490631.0,4670.0,53.0,285.0,3182.0,1150.0
2512,2005.0,183.147500,"Atlanta, GA",430666.0,7213.0,90.0,223.0,4039.0,2861.0
2514,2005.0,198.544167,"Austin, TX",693019.0,3393.0,26.0,312.0,1873.0,1182.0
2516,2005.0,225.512500,"Baltimore, MD",641097.0,11248.0,269.0,162.0,6907.0,3910.0
2518,2005.0,272.906667,"Boston, MA",567589.0,7479.0,73.0,268.0,4489.0,2649.0
...,...,...,...,...,...,...,...,...,...
3468,2015.0,226.920625,"Tampa, FL",364383.0,2298.0,34.0,77.0,1516.0,671.0
3470,2015.0,198.328750,"Tucson, AZ",529675.0,3472.0,31.0,422.0,1960.0,1059.0
3472,2015.0,194.641250,"Tulsa, OK",401520.0,3628.0,55.0,365.0,2354.0,854.0
3474,2015.0,248.035833,"Washington, DC",672228.0,8084.0,162.0,494.0,4024.0,3404.0


In [293]:
crimedata_filter['Crime Rate'] = (
    crimedata_filter[['Violent Crimes', 'Homicides', 'Rapes', 'Assaults', 'Robberies']].sum(axis=1)
    / crimedata_filter['Population']
)

/var/folders/_k/twyqm8dd3gvdw3vfzr0hk_5r0000gn/T/ipykernel_66108/1644214614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crimedata_filter['Crime Rate'] = (


In [294]:
crimedata_filter

,Year,index_nsa,"City, State",Population,Violent Crimes,Homicides,Rapes,Assaults,Robberies,Crime Rate
2510,2005.0,171.870000,"Albuquerque, NM",490631.0,4670.0,53.0,285.0,3182.0,1150.0,0.019037
2512,2005.0,183.147500,"Atlanta, GA",430666.0,7213.0,90.0,223.0,4039.0,2861.0,0.033497
2514,2005.0,198.544167,"Austin, TX",693019.0,3393.0,26.0,312.0,1873.0,1182.0,0.009792
2516,2005.0,225.512500,"Baltimore, MD",641097.0,11248.0,269.0,162.0,6907.0,3910.0,0.035090
2518,2005.0,272.906667,"Boston, MA",567589.0,7479.0,73.0,268.0,4489.0,2649.0,0.026354
...,...,...,...,...,...,...,...,...,...,...
3468,2015.0,226.920625,"Tampa, FL",364383.0,2298.0,34.0,77.0,1516.0,671.0,0.012613
3470,2015.0,198.328750,"Tucson, AZ",529675.0,3472.0,31.0,422.0,1960.0,1059.0,0.013110
3472,2015.0,194.641250,"Tulsa, OK",401520.0,3628.0,55.0,365.0,2354.0,854.0,0.018071
3474,2015.0,248.035833,"Washington, DC",672228.0,8084.0,162.0,494.0,4024.0,3404.0,0.024051


In [295]:
pivot_cd = crimedata_filter.pivot(index='City, State', columns='Year', values='Crime Rate')
pivot_cd.reset_index(inplace=True)
uniquecities = df_income_cr["City"].unique()


In [296]:
df_cr = pivot_cd[pivot_cd["City, State"].isin(uniquecities)]
df_cr.reset_index(inplace=True)
df_cr= df_cr.drop(columns=['index'], errors='ignore')
df_income_cr.reset_index(inplace=True)
df_income_cr= df_income_cr.drop(columns=['index'], errors='ignore')
df_unem_cr.reset_index(inplace=True)
df_unem_cr= df_unem_cr.drop(columns=['index'], errors='ignore')
df_age_cr.reset_index(inplace=True)
df_age_cr= df_age_cr.drop(columns=['index'], errors='ignore')

In [298]:
houseprices = pd.read_csv("../data/house_price.csv") 
df_house = houseprices[houseprices["RegionName"].isin(uniquecities)]

In [299]:
housedata = df_house[["RegionName", "2005-01-31", "2006-01-31", "2007-01-31", "2008-01-31", "2009-01-31", "2010-01-31", "2011-01-31", "2012-01-31", "2013-01-31", "2014-01-31", "2015-01-31"]]
housedata = housedata.sort_values(by='RegionName')
housedata.reset_index(inplace=True)
housedata= housedata.drop(columns=['index'], errors='ignore')

In [300]:
housedata
housedata.columns = [col[:4] for col in housedata.columns]

In [301]:
housedata.rename(columns={'Regi': 'City'}, inplace=True)

In [302]:
housedata

,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,"Albuquerque, NM",153134.096654,170259.007619,193305.919721,200215.211429,187486.929321,180903.756614,171329.029725,164757.245024,163877.783060,166668.565317,169546.379112
1,"Atlanta, GA",179505.707068,186545.601636,192300.481437,191047.167478,171838.828357,152781.210858,139138.933143,125489.469946,131061.115550,154395.394836,169023.933257
2,"Austin, TX",186656.878020,192941.385211,199709.993663,207084.238475,201617.906762,198154.577123,192188.468231,190162.573456,201536.716014,223587.270799,248561.105165
3,"Baltimore, MD",240788.263853,287923.885227,308939.030866,304359.493716,278858.068520,259095.858290,245870.769189,232832.506267,240968.189706,253533.286578,259239.082824
4,"Boston, MA",382715.968493,398584.115753,377807.712517,362087.048748,335987.747593,329631.904785,328532.876118,314696.995585,329627.266928,356868.373186,375171.911811
5,"Buffalo, NY",106986.206474,113148.247931,118401.989443,120363.513874,121148.312897,123696.364041,124964.863241,122728.506796,124598.552361,130046.760989,133706.550598
6,"Charlotte, NC",158158.778134,163575.501056,171928.977411,179513.455323,171800.455853,163543.225698,154086.630300,145167.451962,149210.006802,160115.642609,167340.161683
7,"Chicago, IL",225717.587667,246012.887969,255288.262533,250497.663970,223765.066690,201582.842850,186465.252079,170878.653276,167263.694859,186017.903258,196689.317741
8,"Cincinnati, OH",152092.496851,156955.395939,159202.015388,156781.664959,149453.601401,145555.410360,141293.057061,134287.134103,135560.421066,141983.105910,146468.346256
9,"Cleveland, OH",142794.303419,147024.694658,146576.027152,141066.642378,132698.256834,127238.537647,121999.912014,114934.093129,116848.595250,120420.795953,122229.454170


In [303]:
uniquecities = housedata["City"].unique()
d2 = df_income_cr
d3 = df_unem_cr
d4 = df_age_cr
df_cr = pivot_cd[pivot_cd["City, State"].isin(uniquecities)]
df_cr.reset_index(inplace=True)
df_cr= df_cr.drop(columns=['index'], errors='ignore')
df_income_cr = d2[d2["City"].isin(uniquecities)]
df_income_cr.reset_index(inplace=True)
df_income_cr= df_income_cr.drop(columns=['index'], errors='ignore')
df_unem_cr = d3[d3["City"].isin(uniquecities)]
df_unem_cr.reset_index(inplace=True)
df_unem_cr= df_unem_cr.drop(columns=['index'], errors='ignore')
df_age_cr = d4[d4["City"].isin(uniquecities)]
df_age_cr.reset_index(inplace=True)
df_age_cr= df_age_cr.drop(columns=['index'], errors='ignore')

In [306]:
df_cr.to_csv('../data/crime_rate_35cities.csv')
df_income_cr.to_csv('../data/income_35cities.csv')
df_unem_cr.to_csv('../data/unemployment_35cities.csv')
df_age_cr.to_csv('../data/medianage_35cities.csv')
housedata.to_csv('../data/houseprice_35cities.csv')